# Model

In [1]:
import os
import sys

import pathlib

notebook_path = pathlib.Path(os.getcwd())
sys.path.append(str(notebook_path.parent))

In [2]:
import numpy as np
import pandas as pd

import pickle

from sklearn.compose import ColumnTransformer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import (
    StandardScaler,
    OneHotEncoder
)
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import (
    cross_val_score,
    GridSearchCV,
    train_test_split
) 
from category_encoders import TargetEncoder

from IPython.display import HTML

from database import engine

## Precalculated tables

Sometimes it's too long to do the calculation at runtime. So in some cases we can prepare pre-computed tables. Here is code to create such tables.

**Note** It is stored in the database in order to be used in production.

TF-IDF is an extremely long operation, but fortunately the post data is constant, so we can pre-compute and store TF-IDF vectors.

## Loading data

- We only load records that belong to the `view` category, because every like starts with view, and we have a `target` column that marks those views that lead to likes.

In [47]:
user_data = pd.read_sql(
    "SELECT * FROM public.user_data;",
    con = engine,
    index_col = "user_id"
)
post_data = pd.read_sql(
    "SELECT * FROM public.kobfedsur_post_features_lesson_22;",
    con = engine,
    index_col = "post_id"
)
feed_data = pd.read_sql(
    """
    SELECT
        timestamp,
        user_id,
        post_id,
        action,
        target
    FROM public.feed_data 
    WHERE action='view' 
    LIMIT 200000;
    """,
    con = engine
)

In [48]:
df_show = {
    "Users data" : user_data,
    "Post data" : post_data,
    "Feed data" : feed_data
}

for title, df in df_show.items():
    display(HTML(f"<h3>{title}</h3>"))
    display(df.head())

,gender,age,country,city,exp_group,os,source
user_id,,,,,,,
200,1,34,Russia,Degtyarsk,3,Android,ads
201,0,37,Russia,Abakan,0,Android,ads
202,1,17,Russia,Smolensk,4,Android,ads
203,0,18,Russia,Moscow,1,iOS,ads
204,0,36,Russia,Anzhero-Sudzhensk,3,Android,ads


,topic,text tf_idf0,text tf_idf1,text tf_idf2,text tf_idf3,text tf_idf4,text tf_idf5,text tf_idf6,text tf_idf7,text tf_idf8,...,text tf_idf20,text tf_idf21,text tf_idf22,text tf_idf23,text tf_idf24,text tf_idf25,text tf_idf26,text tf_idf27,text tf_idf28,text tf_idf29
post_id,,,,,,,,,,,,,,,,,,,,,
1,business,0.281532,0.039557,0.037302,0.077854,0.000000,0.036374,0.000000,0.0,0.130329,...,0.068792,0.199227,0.066802,0.763894,0.0,0.000000,0.248880,0.037155,0.144869,0.000000
2,business,0.203882,0.190977,0.030015,0.093968,0.122968,0.058537,0.194172,0.0,0.157303,...,0.276769,0.120231,0.026876,0.657058,0.0,0.027405,0.333768,0.029897,0.194282,0.028149
3,business,0.311137,0.058289,0.274829,0.114722,0.028149,0.026799,0.118528,0.0,0.072017,...,0.152053,0.073392,0.172262,0.737480,0.0,0.000000,0.162993,0.082124,0.000000,0.103097
4,business,0.148869,0.069723,0.000000,0.000000,0.000000,0.000000,0.141779,0.0,0.000000,...,0.121254,0.175579,0.058873,0.742863,0.0,0.000000,0.194967,0.130978,0.170231,0.000000
5,business,0.191478,0.179359,0.084567,0.088252,0.086615,0.000000,0.273539,0.0,0.073867,...,0.077979,0.338749,0.000000,0.597179,0.0,0.000000,0.376155,0.000000,0.000000,0.000000


,timestamp,user_id,post_id,action,target
0,2021-11-13 14:54:07,117300,752,view,0
1,2021-11-13 14:55:54,117300,2293,view,0
2,2021-11-13 14:57:22,117300,5285,view,0
3,2021-11-13 14:58:19,117300,2547,view,0
4,2021-11-13 14:58:58,117300,4152,view,0


## Data preparation

In [17]:
joined_data = pd.merge(
    left = feed_data,
    right = user_data,
    left_on = "user_id",
    right_index = True
)
joined_data = pd.merge(
    left = joined_data,
    right = post_data,
    left_on = "post_id",
    right_index = True
)

In [6]:
X = joined_data.drop("user_id", axis = 1).copy()

y = X["target"]
X.drop("target", axis = 1, inplace = True)

X["month"] = X["timestamp"].dt.month
X["year"] = X["timestamp"].dt.year
X["hour"] = X["timestamp"].dt.hour
X.drop("timestamp", axis = 1, inplace = True)

X['gender'] = X['gender'].astype("O")

In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size = 0.8, random_state = 1
)

## Model pipeline

In [8]:
numeric_columns = list(X_train.select_dtypes("number").columns)
categorical_columns = [
    'gender', 'country', 'city', 'os', 'source', 'topic'
]
cat_nunique = X_train[categorical_columns].nunique()
MTE_columns = cat_nunique.index[cat_nunique > 5].to_list()
OHE_columns = list(set(categorical_columns) - set(MTE_columns))
text_columns = 'text'

In [9]:
data_transformer = ColumnTransformer([
    ("stand_scaler", StandardScaler(), numeric_columns),
    (
        "one_hot_encod", 
        OneHotEncoder(
            categories = list(X_train[OHE_columns].apply(
                lambda col: list(col.unique()),
                result_type="reduce"
            ))
        ), 
        OHE_columns
    ),
    (
        "mean_target_encoder", 
        TargetEncoder(min_samples_leaf = 0.5, smoothing = 0.5), 
        MTE_columns
    ),
    ("tf_idf_vector", TfidfVectorizer(max_features = 10), text_columns)
])

In [10]:
pipeline = Pipeline([
    ("transfmer", data_transformer),
    ("model", GradientBoostingClassifier())
])

## Model selection

In [15]:
grid_search_result = pickle.load(open("gs_results.pck", "rb"))
gs_results_frame = pd.DataFrame(grid_search_result.cv_results_["params"])
gs_results_frame["mean_train_score"] = grid_search_result.cv_results_["mean_test_score"]
gs_results_frame["mean_test_score"] = grid_search_result.cv_results_["mean_train_score"]
gs_results_frame.sort_values("mean_train_score", ascending = False)

,model__learning_rate,model__max_depth,model__n_estimators,transfmer__tf_idf_vector__max_features,mean_train_score,mean_test_score
59,0.2,4,70,20,0.642575,0.736534
55,0.2,4,60,20,0.642094,0.730729
83,0.3,3,70,20,0.641134,0.719360
79,0.3,3,60,20,0.641129,0.715228
82,0.3,3,70,15,0.640824,0.719025
...,...,...,...,...,...,...
34,1.5,5,70,15,0.596681,0.784727
31,1.5,5,60,20,0.594873,0.779732
35,1.5,5,70,20,0.594765,0.786577
33,1.5,5,70,10,0.593987,0.773435


In [16]:
grid_search_result = pickle.load(open("gs_results.pck", "rb"))

best_model = pipeline.set_params(
    **grid_search_result.best_params_
).fit(X_train, y_train)

file_name = "model.pck"
pickle.dump(best_model, open(file_name, "wb"))

## Hitrage@5 estimation

In [17]:
preds_test = pd.Series(
    best_model.predict_proba(X_test)[:,1],
    index = y_test.index
)

data_for_estimation = pd.DataFrame({
    "preds" : preds_test,
    "y" : y_test,
    "user_id" : joined_data.loc[y_test.index, "user_id"]
})

hit_rages = (
    data_for_estimation.
    groupby("user_id").
    apply(lambda x: x.nlargest(5, "preds")["y"].any() if len(x) >= 5 else None)
).dropna().astype("int32")

print("current hitrage@5", hit_rages.mean())

current hitrage@5 0.5002478929102627
